# Data Processing on Google Earth Engine

This notebook shows...
- data processing on Google Earth Engine
- exporting to GCP as TensorFlow objects

# 1. Config

In [20]:
import ee
import geopandas as gpd
import tensorflow as tf
import folium

In [21]:
ee.Initialize()

In [22]:
# Required to export images
# When we export an image, we export everything within a pre-defined region
# We just set the region to be the world (the below)
# Note that we aren't sure why we have to use 88 instead of 90 - might have come from Sara!
GEOMETRY_WORLD = ee.Geometry.Polygon(
    [
        [-180, 88],
        [180, 88],
        [180, -88],
        [-180, -88],
    ],
    None,
    False
)

In [23]:
# Creating add_ee_layer function
# This allows us to visualise EE objects on a Folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee_image_object.getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# 2. Data Processing

## 2.1. Fault labels

Here we will turn the vectors into a raster (image of 1s and 0s where 1 indicates a fault and 0 indicates no fault).

In [24]:
# Read in data from EE
faultline_vectors = ee.FeatureCollection("projects/esg-satelite/assets/mars/labels/faults/pre/faults")

In [25]:
# Add a columns of ones 
# This assigns each geometry (each fault/vector) to the value 1
# and assigns every other space to nothing (which we will fix later)
def add_column(feature):
    return feature.set({"Value": 1})

faultline_vectors_with_ones = faultline_vectors.map(add_column)

In [26]:
faultline_im = (
    # Convert to image
    faultline_vectors_with_ones.reduceToImage(
        properties=["Value"],
        reducer=ee.Reducer.first()
    )
    # Change band name "first"  to "FAULTLINE" 
    .select(["first"], ["FAULTLINE"])
    # Unmask, which changes all non-fault pixels to have value 0
    .unmask(0)
)

In [27]:
# Export to EE
# NOTE: There is a crs parameter that can be assigned here
# NOTE: We can play around with different scales (scale might require the crs being assigned)
task = ee.batch.Export.image.toAsset(
    faultline_im,
    description = "Faultline_image",
    assetId="projects/esg-satelite/assets/mars/labels/faults/post/faults_raster",
    region=GEOMETRY_WORLD,
    scale=500,
    maxPixels=1e11 
    # TODO: There is a crs parameter that can be assigned here?
)

In [28]:
task.start()

In [29]:
# Trying another pixel size
task = ee.batch.Export.image.toAsset(
    faultline_im,
    description = "Faultline_image",
    assetId="projects/esg-satelite/assets/mars/labels/faults/post/faults_raster_250",
    region=GEOMETRY_WORLD,
    scale=250,
    maxPixels=1e11 
    # TODO: There is a crs parameter that can be assigned here?
)

In [30]:
task.start()

## 2.2. HRSC Images (sample of 3 only)

Here we will do two things
- Remove the black band from each image
- Mosaic the images together (i.e. flatten them into one image)

In [ ]:
# Read in data from EE
mars_hrsc_sample = ee.ImageCollection("projects/esg-satelite/assets/mars/features/hrsc_sample_3/pre/hrsc_sample")

In [ ]:
# Renaming the band from "b1" to HRSC
mars_hrsc_sample = mars_hrsc_sample.select(["b1"], ["HRSC"])

In [ ]:
# Removes black band
# Applies a mask that keeps only the parts that have value greater than ("gt") 0
# Note that black is 0
def pixel_mask(image):
    return image.updateMask((image.select("HRSC").gt(0)))

mars_hrsc_filtered = mars_hrsc_sample.map(pixel_mask)

In [ ]:
# Mosaics the image collection
mars_hrsc_mosaic = mars_hrsc_filtered.mosaic()

In [ ]:
# This is the code we used to check whether the black bands were actually 0
# TODO: What does this code do again? Forgotton!
# region = mars_hrsc_sample.geometry()
# maxReducer = ee.Reducer.min()
# print(mars_hrsc_mosaic.reduceRegion(maxReducer, region, 300, bestEffort=True).getInfo())

In [ ]:
# Export to EE
# NOTE: There is a crs parameter that can be assigned here
# NOTE: We can play around with different scales (scale might require the crs being assigned)
task = ee.batch.Export.image.toAsset(
    mars_hrsc_mosaic,
    description = "HRSC_Image",
    assetId="projects/esg-satelite/assets/mars/features/hrsc_sample_3/post/hrsc_sample",
    region=GEOMETRY_WORLD,
    scale=1000,
    maxPixels=1e11
)

In [ ]:
task.start()

## 2.3. CTX (Sample of 2 only)

In [ ]:
# Read in data from EE
ctx_sample = ee.ImageCollection("projects/esg-satelite/assets/mars/features/ctx_sample_2/pre/ctx_sample")

In [ ]:
# Printing band name
ctx_sample.first().bandNames().getInfo()

In [ ]:
# Renaming the band from "b1" to CTX
ctx_sample = ctx_sample.select(["b1"], ["CTX"])

In [ ]:
faultline_vectors = ee.FeatureCollection("projects/esg-satelite/assets/mars/labels/faults/pre/faults")

Map = folium.Map()

Map.add_ee_layer(
    ctx_sample,
    {},
    "CTX"
)

Map.add_ee_layer(
    faultline_vectors,
    {"color":"red"},
    "faults"
)

_ = folium.LayerControl().add_to(Map)

Map

## 2.5. CTX Tempe Terra

In [ ]:
# Read in data from EE
ctx_tempe_terra = ee.ImageCollection("projects/esg-satelite/assets/mars/features/ctx_tempe_terra/pre/ctx_tempe_terra")

In [ ]:
Map = folium.Map()

Map.add_ee_layer(
    ctx_tempe_terra,
    {},
    "CTX Tempe Terra"
)

Map.add_ee_layer(
    faultline_vectors,
    {"color":"red"},
    "faults"
)

_ = folium.LayerControl().add_to(Map)

Map

## 2.6. THEMIS Mosaic

In [ ]:
# Read in data from EE
themis_mosaic = ee.Image("projects/esg-satelite/assets/mars/features/themis_mosaic/pre/themis_mosaic")

In [ ]:
themis_sample = ee.ImageCollection("projects/esg-satelite/assets/mars/features/themis_sample_4/pre/themis_sample")

In [ ]:
ctx_tempe_terra = ee.ImageCollection("projects/esg-satelite/assets/mars/features/ctx_tempe_terra/pre/ctx_tempe_terra")
faultline_vectors = ee.FeatureCollection("projects/esg-satelite/assets/mars/labels/faults/pre/faults")
hrsc_sample = ee.Image("projects/esg-satelite/assets/mars/features/hrsc_sample_3/post/hrsc_sample")

In [ ]:
tt_faultclip_binary = ee.Image("projects/esg-satelite/assets/mars/test/TT_fault_clip_binary")

In [ ]:
Map = folium.Map() # crs = "EPSG4326"

Map.add_ee_layer(
    ctx_tempe_terra,
    {},
    "CTX Tempe Terra"
)

Map.add_ee_layer(
    themis_mosaic,
    {},
    "THEMIS Mosaic"
)

Map.add_ee_layer(
    themis_sample,
    {},
    "THEMIS Sample"
)


Map.add_ee_layer(
    hrsc_sample,
    {},
    "HRSC Sample"
)

Map.add_ee_layer(
    faultline_vectors,
    {"color":"red"},
    "faults"
)

# Map.add_ee_layer(
#     tt_faultclip_binary,
#     {},
#     "TT Fault Clip Binary"
# )

_ = folium.LayerControl().add_to(Map)

Map

## 2.5. THEMIS 4 Samples

In [ ]:
themis_samples = ee.ImageCollection("projects/esg-satelite/assets/mars/features/themis_sample_4/pre/themis_sample")

In [ ]:
Map = folium.Map(crs = )

Map.add_ee_layer(
    themis_samples,
    {},
    "THEMIS Samples (EE)"
)

Map.add_ee_layer(
    faultline_vectors,
    {"color":"red"},
    "faults"
)

_ = folium.LayerControl().add_to(Map)

Map

In [ ]:
import json

In [ ]:
themis_mosaic.projection().getInfo()

In [ ]:
print(themis_mosaic.projection())